### 1. 引入工具包

In [1]:
import pandas as pd

import numpy as np

In [3]:
from pymining import itemmining, assocrules, perftesting

In [11]:
from orangecontrib.associate import slpgrowth

### 2. 数据预处理

In [2]:
raw_df = pd.read_csv('./pj/rawData1.CSV', dtype=str)

#删除无关列
del raw_df['NO']

In [3]:
raw_df.head()

,Gender,AloneChild,Relationship,NumChildren,Parents,Relatives,Title,ProjectType,ProjectLoc,Position,...,Group_AFC,Group_ACC,Group_LA,Group_NC,WVType,Group_M_GoodTimes,Group_M_Cognitive,Group_M_WorkTeam,Group_M_Instrum,Group_M_Altrui
0,1,1,1,0,0,0,0,NaN,3,1,...,2,2,2,2,22,1,2,2,1,1
1,1,1,1,0,0,0,3,1,1,1,...,3,3,2,3,22,1,1,1,2,1
2,1,1,3,0,1,0,4,1,2,1,...,2,2,2,2,6,1,2,2,2,2
3,1,0,3,2,1,0,4,1,2,3,...,3,2,3,3,19,2,2,3,2,3
4,0,1,1,0,0,1,4,1,2,1,...,3,2,2,3,3,2,1,3,2,1


In [ ]:
# 查看有缺失值的columns
# nan_col = pd.DataFrame(raw_df.isnull().any(), columns={'is_nan'})
# nan_col['col'] = nan_col.index
# nan_col_list = list(nan_col[nan_col['is_nan']==True]['col'])
# nan_col_list

#用mode填充缺失值
# pna_df = raw_df.copy()
# for col in nan_col_list:
#     pna_df[col].fillna(pna_df[col].mode()[0], inplace=True)

In [7]:
ConsItem = ['Category_VI', 'Category_DE', 'Category_AB', 'Group_TP', 'Group_IF', 'Group_AFC', 'Group_ACC', 'Group_LA', 'Group_NC']

Supplies = ['Group_J_SAL', 'Group_J_BEN', 'Group_J_ADV', 'Group_J_FAR', 'Group_J_SSU', 'Group_J_CSU', 'Group_J_COW', 'Group_J_COO', 'Group_J_CHA', 'Group_J_CRE', 'Group_J_IST', 'Group_J_CLN', 'Group_J_TRG', 'Group_J_HRS', 'Group_J_BAL', 'Group_J_FRE', 'Group_J_FBK', 'Group_J_IND', 'Group_J_ABI', 'Group_J_INT', 'Group_J_VAR', 'Group_J_FUN', 'Group_J_ENV', 'Group_J_COS', 'Group_J_HLP', 'Group_J_IFL', 'Group_J_IMP']

WorkValues1 = ['Group_SAL', 'Group_BEN', 'Group_ADV', 'Group_FAR', 'Group_SSU', 'Group_CSU', 'Group_COW', 'Group_COO', 'Group_CHA', 'Group_CRE', 'Group_IST', 'Group_CLN', 'Group_TRG', 'Group_HRS', 'Group_BAL', 'Group_FRE', 'Group_FBK', 'Group_IND', 'Group_ABI', 'Group_INT', 'Group_VAR', 'Group_FUN', 'Group_ENV', 'Group_COS', 'Group_HLP', 'Group_IFL', 'Group_IMP']

WorkValues2 = ['Group_M_GoodTimes', 'Group_M_Cognitive', 'Group_M_WorkTeam', 'Group_M_Instrum', 'Group_M_Altrui']

WorkValues3 = ['WVType']

PsJc = ['Gender', 'AloneChild', 'Relationship', 'NumChildren', 'Parents', 'Relatives', 'Title', 'ProjectType', 'ProjectLoc', 'Position', 'DE24_1', 'DE24_2', 'DE24_3', 'DE24_5', 'DE24_6', 'DE24_7', 'DE24_8', 'DE24_9', 'GrowthEnv', 'Edu', 'Contracting', 'Ownership', 'AgeGroup', 'CPMPyrs', 'Org_yrs']

In [16]:
from scipy.sparse import lil_matrix  # other types would convert to LIL anyway

def get_ana_input(raw_df, cols):
    df = raw_df[cols]
    oh_df = pd.get_dummies(df) #one-hot编码
    if 'ProjectType_5' in oh_df.columns.values:
        del oh_df['ProjectType_5'] #删除无效项
    if 'Contracting_4' in oh_df.columns.values:
        del oh_df['Contracting_4']
    oh_df = pd.DataFrame(oh_df, dtype=int) #将属性类型转换为int32
    
    #构造字典记录head的位置
    col_idx = list(range(oh_df.shape[1]))
    col_name = oh_df.columns.values
    ins = zip(col_idx, col_name)
    col_dict = dict((col_idx,col_name) for col_idx, col_name in ins)

    oh_tb = oh_df > .9
    X = lil_matrix(oh_tb.values) #稀疏矩阵
    itme_set = []
    for l in X.rows: #转换成item set
        row = []
        for i in l:
            #row.append(col_dict[i])
            row.append(i)
        itme_set.append(tuple(row))
#     itme_set = tuple(itme_set)
    return itme_set, col_dict

In [17]:
ds_1, col_dic_1 = get_ana_input(raw_df, Supplies + ConsItem)
ds_2, col_dic_2 = get_ana_input(raw_df, Supplies + WorkValues1 + ConsItem)
ds_3, col_dic_3 = get_ana_input(raw_df, Supplies + WorkValues1 + PsJc + ConsItem)
ds_4, col_dic_4 = get_ana_input(raw_df, Supplies + WorkValues2 + ConsItem)
ds_5, col_dic_5 = get_ana_input(raw_df, Supplies + WorkValues2 + PsJc + ConsItem)
ds_6, col_dic_6 = get_ana_input(raw_df, Supplies + WorkValues3 + ConsItem)
ds_7, col_dic_7 = get_ana_input(raw_df, Supplies + WorkValues3 + PsJc + ConsItem)

In [5]:
#生成表单
def rules_to_df(rules):  
    return_rules = []
    for i in rules:
        tem_list = []
        k = '';
        for obj in i[0]: 
            k = k + '[' + obj + ']' + ' & '
        
        k = k[:-3]
        flag = False
        v = ''
        for obj in i[1]:
            if obj[:-2] not in ConsItem:
                flag = True
                break
            v = v + '[' + obj + ']' + ' & '
        if flag:
            continue
        v = v[:-3]
        tem_list.append(k); tem_list.append(v); tem_list.append(i[2]); tem_list.append(i[3]); tem_list.append(i[4])
        tem_list.append(i[5]); tem_list.append(i[6]); tem_list.append(i[7])
        return_rules.append(tem_list)
    return pd.DataFrame(return_rules, columns=('atecedent','consequent','support','confidence','coverage','strength','lift','leverage'))

In [6]:
def get_ass_rules(ds, min_sup = 200, min_conf = 0.3):
    relim_input = itemmining.get_relim_input(ds)
    item_sets = itemmining.relim(relim_input, min_support = min_sup)
    rules = assocrules.mine_assoc_rules(item_sets, min_support = min_sup, min_confidence = min_conf)
    rules_info = list(oaf.rules_stats(rules, item_sets, raw_df.shape[0]))    #统计评估信息
    return rules_to_df(rules_info)

In [6]:
def get_ass_rules(ds, min_sup = 200, min_conf = 0.3):
    relim_input = itemmining.get_relim_input(ds)
    item_sets = itemmining.relim(relim_input, min_support = min_sup)
    rules = assocrules.mine_assoc_rules(item_sets, min_support = min_sup, min_confidence = min_conf)
    rules_info = list(oaf.rules_stats(rules, item_sets, raw_df.shape[0]))    #统计评估信息
    return rules_to_df(rules_info)

In [11]:
ars_1 = get_ass_rules(ds_1)
ars_2 = get_ass_rules(ds_2)
ars_3 = get_ass_rules(ds_3)
ars_4 = get_ass_rules(ds_4)
ars_5 = get_ass_rules(ds_5)
ars_6 = get_ass_rules(ds_6)
ars_7 = get_ass_rules(ds_7)

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [12]:
ars_1.to_csv('ars_1.csv', index=False)
ars_2.to_csv('ars_2.csv', index=False)
ars_3.to_csv('ars_3.csv', index=False)
ars_4.to_csv('ars_4.csv', index=False)
ars_5.to_csv('ars_5.csv', index=False)
ars_6.to_csv('ars_6.csv', index=False)
ars_7.to_csv('ars_7.csv', index=False)

In [15]:
ars_1.shape
ars_2.shape
ars_3.shape
ars_4.shape
ars_5.shape
ars_6.shape
ars_7.shape

(30919, 8)

(157808, 8)

(279117, 8)

(60053, 8)

(135139, 8)

(30924, 8)

(75655, 8)

In [21]:
min_sup = 200
min_conf = 0.3

In [23]:
%%time
relim_input = itemmining.get_relim_input(ds_1)
item_sets = itemmining.relim(relim_input, min_support = min_sup)

Wall time: 4.63 s


In [24]:
%%time
fptree_input = itemmining.get_fptree(ds_1)
item_sets = itemmining.fpgrowth(fptree_input, min_support = min_sup)

Wall time: 3.37 s


In [18]:
%%time
slptree_input = slpgrowth.frequent_itemsets(ds_1, min_support=50, max_support=200)
#item_sets = itemmining.fpgrowth(fptree_input, min_support = min_sup)

Wall time: 0 ns


In [19]:
list(slptree_input)

KeyboardInterrupt: 

In [18]:
len(item_sets)

148969

In [26]:
rules = assocrules.mine_assoc_rules(item_sets, min_support = min_sup, min_confidence = min_conf)

In [27]:
len(rules)

576694

In [28]:
rules_info = list(oaf.rules_stats(rules, item_sets, raw_df.shape[0]))    #统计评估信息


In [33]:
rules_info

[(frozenset({'Group_IF_3',
             'Group_J_ABI_4',
             'Group_J_CLN_4',
             'Group_J_COW_4',
             'Group_J_HLP_4',
             'Group_J_INT_4',
             'Group_TP_3'}),
  frozenset({'Group_J_COS_4'}),
  202,
  0.8595744680851064,
  0.23618090452261306,
  2.4680851063829787,
  1.4746148202494498,
  0.06534178429837631),
 (frozenset({'Group_IF_3',
             'Group_J_ABI_4',
             'Group_J_CLN_4',
             'Group_J_COW_4',
             'Group_J_HLP_4',
             'Group_TP_3'}),
  frozenset({'Group_J_COS_4', 'Group_J_INT_4'}),
  202,
  0.7651515151515151,
  0.26532663316582916,
  1.5075757575757576,
  1.9128787878787876,
  0.09688442211055276),
 (frozenset({'Group_J_ABI_4',
             'Group_J_CLN_4',
             'Group_J_COW_4',
             'Group_J_HLP_4',
             'Group_TP_3'}),
  frozenset({'Group_IF_3', 'Group_J_COS_4', 'Group_J_INT_4'}),
  202,
  0.7345454545454545,
  0.27638190954773867,
  1.3490909090909091,
  1.9700073

In [31]:
a1 = rules_to_df(rules_info)

In [32]:
a1.head()

,atecedent,consequent,support,confidence,coverage,strength,lift,leverage


In [ ]:
relim_input = itemmining.get_relim_input(ds)
item_sets = itemmining.relim(relim_input, min_support = min_sup)

In [52]:
rules

[(frozenset({'Group_IF_3',
             'Group_J_ABI_4',
             'Group_J_CLN_4',
             'Group_J_COW_4',
             'Group_J_HLP_4',
             'Group_J_INT_4',
             'Group_TP_3'}),
  frozenset({'Group_J_COS_4'}),
  202,
  0.8595744680851064),
 (frozenset({'Group_IF_3',
             'Group_J_ABI_4',
             'Group_J_CLN_4',
             'Group_J_COW_4',
             'Group_J_HLP_4',
             'Group_TP_3'}),
  frozenset({'Group_J_COS_4', 'Group_J_INT_4'}),
  202,
  0.7651515151515151),
 (frozenset({'Group_J_ABI_4',
             'Group_J_CLN_4',
             'Group_J_COW_4',
             'Group_J_HLP_4',
             'Group_TP_3'}),
  frozenset({'Group_IF_3', 'Group_J_COS_4', 'Group_J_INT_4'}),
  202,
  0.7345454545454545),
 (frozenset({'Group_J_ABI_4',
             'Group_J_CLN_4',
             'Group_J_COW_4',
             'Group_J_HLP_4'}),
  frozenset({'Group_IF_3', 'Group_J_COS_4', 'Group_J_INT_4', 'Group_TP_3'}),
  202,
  0.6622950819672131),
 (froz

In [46]:
'abcd'[:-2]

'ab'

In [7]:
ars_1 = pd.read_csv('ars_1.csv')
ars_2 = pd.read_csv('ars_2.csv')
ars_3 = pd.read_csv('ars_3.csv')
ars_4 = pd.read_csv('ars_4.csv')
ars_5 = pd.read_csv('ars_5.csv')
ars_6 = pd.read_csv('ars_6.csv')
ars_7 = pd.read_csv('ars_7.csv')

In [22]:
writer = pd.ExcelWriter('ars.xlsx')
for i in range(1, 8):
    df_name = 'ars_' + str(i)
    df = eval(df_name)
    df.to_excel(excel_writer=writer, sheet_name=df_name, encoding="utf-8", index=False)
writer.save()
writer.close()

In [9]:
ars_1.head()

,atecedent,consequent,support,confidence,coverage,strength,lift,leverage
0,[Group_J_HLP_4] & [Group_J_INT_4] & [Group_J_C...,[Group_IF_3],202,0.980583,0.207035,4.092233,1.157390,0.027607
1,[Group_J_HLP_4] & [Group_J_INT_4] & [Group_J_C...,[Group_TP_3] & [Group_IF_3],202,0.922374,0.220101,3.378995,1.240220,0.039322
2,[Group_J_HLP_4] & [Group_J_INT_4] & [Group_J_C...,[Group_TP_3],202,0.961905,0.211055,3.785714,1.203893,0.034383
3,[Group_J_HLP_4] & [Group_J_INT_4] & [Group_J_C...,[Group_IF_3],205,0.990338,0.208040,4.072464,1.168904,0.029771
4,[Group_J_HLP_4] & [Group_J_INT_4] & [Group_J_I...,[Group_TP_3] & [Group_IF_3],205,0.927602,0.222111,3.348416,1.247248,0.040842


In [32]:
ars_1.shape

(30919, 8)

In [33]:
df = ars_1
df[~df['consequent'].str.contains('_3')].shape

(333, 8)

In [12]:
df = ars_1
# df = df[df['support'] > sup]

In [8]:
def ana_ars(ars_df, min_sup = 200, sup_step = 50, min_conf = 0.3, conf_step = 0.1, min_lift = 1.1):
    df = ars_df
    ars_info = []
    cols = ('support', 'confidence', 'rule_num', 'lift', 'srule_num', 'no2_srule_num', 'no2_srule_lift_avg')
    for sup in range(min_sup, 1000, sup_step):
        one_info = []
        df = df[df['support'] > sup]
        for conf in np.arange(min_conf, 1.0, conf_step):
            df = df[df['confidence'] > conf]
            rule_num = df.shape[0]
            if rule_num == 0:
                break
            df = df[df['lift'] >= min_lift]
            srule_num = df.shape[0]
            df = df[~df['consequent'].str.contains('_2')]
            no2_srule_num = df.shape[0]
            no2_srule_lift_avg = df['lift'].mean()
            one_info = [sup, conf, rule_num, min_lift, srule_num, no2_srule_num, no2_srule_lift_avg]
            ars_info.append(one_info)
    return pd.DataFrame(ars_info, columns=cols)

In [9]:
aa_1 = ana_ars(ars_1)

aa_2 = ana_ars(ars_2)

aa_3 = ana_ars(ars_3)

aa_4 = ana_ars(ars_4)

aa_5 = ana_ars(ars_5)

aa_6 = ana_ars(ars_6)

aa_7 = ana_ars(ars_7)

In [17]:
aaa = 1

In [18]:
eval('aaa')

1

In [20]:
writer = pd.ExcelWriter('ars_ana.xlsx')
for i in range(1, 8):
    df_name = 'aa_' + str(i)
    df = eval(df_name)
    df.to_excel(excel_writer=writer, sheet_name=df_name, encoding="utf-8", index=False)
writer.save()
writer.close()

In [16]:
aa_7

,support,confidence,rule_num,lift,srule_num,no2_srule_num,no2_srule_lift_avg
0,200,0.3,72962,1.1,51960,51771,1.196593
1,200,0.4,51583,1.1,51583,51583,1.196738
2,200,0.5,50892,1.1,50892,50892,1.197098
3,200,0.6,48496,1.1,48496,48496,1.196196
4,200,0.7,41057,1.1,41057,41057,1.185658
5,200,0.8,35933,1.1,35933,35933,1.153778
6,200,0.9,24062,1.1,24062,24062,1.138381
7,250,0.3,3828,1.1,3828,3828,1.133407
8,250,0.4,3828,1.1,3828,3828,1.133407
9,250,0.5,3828,1.1,3828,3828,1.133407


### 3. 关联分析

In [61]:
#最小支持度和置信度
minsup = 0.3
minconf = 0.6

In [62]:
#挖掘频繁项集(包括one itme的set,为了后面处理方便)
itemsets = dict(oaf.frequent_itemsets(X, min_support=minsup))

In [63]:
len(itemsets)

12066

In [35]:
itemsets

{frozenset({397}): 843,
 frozenset({82}): 824,
 frozenset({82, 397}): 693,
 frozenset({1}): 811,
 frozenset({1, 82}): 681,
 frozenset({1, 397}): 689,
 frozenset({1, 82, 397}): 572,
 frozenset({394}): 795,
 frozenset({1, 394}): 647,
 frozenset({82, 394}): 652,
 frozenset({1, 82, 394}): 535,
 frozenset({394, 397}): 740,
 frozenset({1, 394, 397}): 603,
 frozenset({82, 394, 397}): 606,
 frozenset({1, 82, 394, 397}): 497,
 frozenset({14}): 761,
 frozenset({1, 14}): 620,
 frozenset({14, 82}): 617,
 frozenset({1, 14, 82}): 508,
 frozenset({14, 397}): 641,
 frozenset({1, 14, 397}): 524,
 frozenset({14, 82, 397}): 514,
 frozenset({1, 14, 82, 397}): 424,
 frozenset({14, 394}): 591,
 frozenset({1, 14, 394}): 481,
 frozenset({14, 82, 394}): 471,
 frozenset({1, 14, 82, 394}): 385,
 frozenset({14, 394, 397}): 553,
 frozenset({1, 14, 394, 397}): 451,
 frozenset({14, 82, 394, 397}): 440,
 frozenset({1, 14, 82, 394, 397}): 361,
 frozenset({24}): 744,
 frozenset({1, 24}): 602,
 frozenset({24, 394}): 600

In [50]:
t = list(itemsets.keys())

In [61]:
2 in t[1]

False

In [64]:
#挖掘关联规则
rules = list(oaf.association_rules(itemsets, minconf))

In [65]:
len(rules)

58032

In [30]:
#统计评估信息
rules = list(oaf.rules_stats(rules, itemsets, raw_df.shape[0]))

In [31]:
#生成表单
def rules_to_df(rules):  
    return_rules = []
    for i in rules:
        tem_list = []
        tem_str = '';
        for j in i[0]: 
            tem_str = tem_str + '[' + col_dict[j] + ']' + ' & '
        
        tem_str = tem_str[:-2]
        tem_str = tem_str + '  ==>  '
        for j in i[1]:
            tem_str = tem_str + '[' + col_dict[j] + ']' + ' & '
        tem_str = tem_str[:-2]
        tem_list.append(tem_str); tem_list.append(i[2]); tem_list.append(i[3]); tem_list.append(i[4])
        tem_list.append(i[5]); tem_list.append(i[6]); tem_list.append(i[7])
        return_rules.append(tem_list)
    return pd.DataFrame(return_rules,columns=('rules','support','confidence','coverage','strength','lift','leverage'))


rules_df = rules_to_df(rules)


In [32]:
target_item = ['Category_VI', 'Category_DE', 'Category_AB', 'Group_TP', 'Group_IF', 'Group_AFC', 'Group_ACC', 'Group_LA', 'Group_NC']

In [33]:
'Category_VI_3'[:-2] in target_item

True

In [34]:
lr_dict = dict()
rl_dict = dict()

In [444]:
#生成表单
def tgt_rules_to_df(rules):  
    return_rules = []
    for i in rules:
        tem_list = []
        k = '';
        for j in i[0]: 
            k = k + '[' + col_dict[j] + ']' + ' & '
        
        k = k[:-2]
        flag = False
        v = ''
        for j in i[1]:
            if col_dict[j][:-3] not in target_item:
                flag = True
                break
            v = v + '[' + col_dict[j] + ']' + ' & '
        if flag:
            continue
        v = v[:-2]
        lr_dict[k] = v
        rl_dict[v] = k
        tem_str = k + '  ==>  ' + v
        tem_list.append(k); tem_list.append(v); tem_list.append(i[2]); tem_list.append(i[3]); tem_list.append(i[4])
        tem_list.append(i[5]); tem_list.append(i[6]); tem_list.append(i[7])
        return_rules.append(tem_list)
    return pd.DataFrame(return_rules,columns=('atecedent','consequent ','support','confidence','coverage','strength','lift','leverage'))

tgt_rules_df = tgt_rules_to_df(rules)


In [445]:
print("------------"+ "minsup: " + str(minsup) + "; minconf: " + str(minconf) + "; rules_num: " + str(tgt_rules_df.shape[0]) + "------------")
tgt_rules_df.head()

------------minsup: 0.3; minconf: 0.6; rules_num: 8529------------


,atecedent,consequent,support,confidence,coverage,strength,lift,leverage
0,[Group_J_COW_3 ] & [Group_IF_2 ] & [Group_J_SS...,[Group_TP_2 ],304,0.910180,0.335678,2.380240,1.139156,0.037322
1,[Group_J_COW_3 ] & [Group_TP_2 ] & [Group_J_SS...,[Group_IF_2 ],304,0.955975,0.319598,2.650943,1.128345,0.034753
2,[Group_J_SSU_3 ] & [Group_J_COW_3 ] & [Group_J...,[Group_TP_2 ] & [Group_IF_2 ],304,0.832877,0.366834,2.027397,1.119882,0.032706
3,[Group_J_COW_3 ] & [Group_TP_2 ] & [Group_J_SS...,[Group_IF_2 ],299,0.949206,0.316583,2.676190,1.120356,0.032282
4,[Group_J_SSU_3 ] & [Group_J_COW_3 ] & [Group_J...,[Group_TP_2 ] & [Group_IF_2 ],299,0.832869,0.360804,2.061281,1.119871,0.032166


In [414]:
print("------------"+ "minsup: " + str(minsup) + "; minconf: " + str(minconf) + "; rules_num: " + str(rules_df.shape[0]) + "------------")
rules_df.head()

------------minsup: 0.3; minconf: 0.6; rules_num: 58032------------


,rules,support,confidence,coverage,strength,lift,leverage
0,[Group_TP_2 ] & [Group_IF_2 ] & [Group_J_SSU_3...,304,0.891496,0.342714,1.976540,1.316080,0.073378
1,[Group_J_CSU_3 ] & [Group_TP_2 ] & [Group_J_CL...,304,0.769620,0.396985,1.278481,1.516380,0.104043
2,[Group_J_SSU_3 ] & [Group_TP_2 ] & [Group_J_CL...,304,0.787565,0.387940,1.347150,1.506975,0.102785
3,[Group_J_COW_3 ] & [Group_TP_2 ] & [Group_IF_2...,304,0.894118,0.341709,1.800000,1.453672,0.095351
4,[Group_J_COW_3 ] & [Group_IF_2 ] & [Group_J_SS...,304,0.910180,0.335678,2.380240,1.139156,0.037322


In [415]:
rules_df[rules_df['lift']>1].sort_values(by='lift', ascending=False)

,rules,support,confidence,coverage,strength,lift,leverage
57931,[Category_WE_2 ] ==> [Category_DE_2 ],311,0.740476,0.422111,0.959524,1.828223,0.141597
57932,[Category_DE_2 ] ==> [Category_WE_2 ],311,0.771712,0.405025,1.042184,1.828223,0.141597
30655,[Group_HLP_3 ] & [Group_CHA_3 ] ==> [Group_...,303,0.740831,0.411055,1.004890,1.793497,0.134730
30659,[Group_COS_3 ] & [Group_IST_3 ] ==> [Group_...,303,0.737226,0.413065,0.995134,1.793497,0.134730
31923,[Group_CRE_3 ] & [Group_CSU_3 ] ==> [Group_...,304,0.785530,0.388945,1.147287,1.760365,0.131968
31921,[Group_SSU_3 ] & [Group_CHA_3 ] ==> [Group_...,304,0.684685,0.446231,0.871622,1.760365,0.131968
7127,[Group_CRE_3 ] & [Group_CLN_3 ] & [Group_IF_2 ...,321,0.829457,0.388945,1.219638,1.748538,0.138109
7118,[Group_CHA_3 ] & [Group_IST_3 ] ==> [Group_...,321,0.680085,0.474372,0.819915,1.748538,0.138109
7098,[Group_CHA_3 ] & [Group_IST_3 ] ==> [Group_...,306,0.648305,0.474372,0.781780,1.748140,0.131615
7084,[Group_CRE_3 ] & [Group_CLN_3 ] & [Group_TP_2 ...,306,0.829268,0.370854,1.279133,1.748140,0.131615


In [416]:
rules_df.to_excel('regular.xlsx')

In [446]:
tgt_rules_df.to_excel('tgt_rules_df.xlsx')